<a href="https://colab.research.google.com/github/wasimmadha/dubbing-project-research/blob/main/librspeech_dataset_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!tar -xzf '/content/drive/MyDrive/Dubbing Project/libspeech/dev-clean.tar.gz' -C /content/

In [8]:
!tar -xzf '/content/drive/MyDrive/Dubbing Project/libspeech/train-clean-100.tar.gz' -C /content/

In [ ]:
import os

output_dict = {}
main_folder_path = '/content/LibriSpeech/train-clean-100'

# for main_speaker_dir in os.listdir()